In [1]:
import numpy as np
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir) 
from main_grape.Grape import Grape
from helper_functions.grape_functions import *
%pylab inline




Populating the interactive namespace from numpy and matplotlib


/usr/local/lib/python2.7/dist-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:

#Defining time scales

qubit_num = 7
steps = 200*qubit_num
total_time = 1.5*qubit_num
qubit_state_num = 2

       

#Defining Forbidden sates

#states_forbidden_list = []

#Defining Concerned states (starting states)
psi0 = concerned(qubit_num,qubit_state_num)

#Defining states to include in the drawing of occupation
states_draw_list = range(qubit_state_num**qubit_num)
states_draw_names =[]
for ii in states_draw_list:
    states_draw_names.append(Basis(ii,qubit_num,qubit_state_num))

#Defining U (Target)

U= transmon_gate(qft(qubit_num),qubit_state_num)
#U = np.diag(np.arange(0,4))

#Defining U0 (Initial)
qq_identity = np.identity(qubit_state_num**qubit_num)
U0= qq_identity

#Defining control Hs
Q_x   = np.diag(np.sqrt(np.arange(1,qubit_state_num)),1)+np.diag(np.sqrt(np.arange(1,qubit_state_num)),-1)
Q_y   = (0+1j) *(np.diag(np.sqrt(np.arange(1,qubit_state_num)),1)-np.diag(np.sqrt(np.arange(1,qubit_state_num)),-1))
Q_z   = np.diag(np.arange(0,qubit_state_num))
Q_I = np.identity(qubit_state_num)

Q_xx = multi_kron(Q_x,qubit_num)
Q_yy = multi_kron(Q_y,qubit_num)
Q_zz = multi_kron(Q_z,qubit_num)

H0 = 0.5* (Q_xx+Q_yy+Q_zz)

Hops = []
Hnames = []
ops_max_amp = []
Hops,Hnames,ops_max_amp = append_separate_krons(Q_x,'x',qubit_num,qubit_state_num,Hops,Hnames,ops_max_amp,amp=6.0)
Hops,Hnames,ops_max_amp = append_separate_krons(Q_y,'y',qubit_num,qubit_state_num,Hops,Hnames,ops_max_amp,amp=6.0)
Hops,Hnames,ops_max_amp = append_separate_krons(Q_z,'z',qubit_num,qubit_state_num,Hops,Hnames,ops_max_amp,amp=6.0)
Hops.extend([Q_xx,Q_yy,Q_zz])
Hnames.extend(['all x','all y','all z'])
ops_max_amp.extend([6.0,6.0,6.0])

#Defining convergence parameters
max_iterations = 5000
decay = max_iterations/2
convergence = {'rate':0.05, 'update_step':100, 'max_iterations':max_iterations,\
               'conv_target':1e-3,'learning_rate_decay':decay}
penalty_coeffs = {'envelope' : 0, 'dc':0, 'dwdt':0,'d2wdt2':0, 'forbidden':0}
#multi_mode = {'dressed':dressed, 'vectors':v_c, 'qnum':qubit_state_num, 'mnum': mode_state_num,\
              #'f':freq_ge, 'es':w_c, 'g1':qm_g1, 'D':D, 'Interpolation':True, 'Modulation':True}

print states_draw_names
print Hnames
print np.shape(U)
print np.shape(Hops)





['0000000', '0000001', '0000010', '0000011', '0000100', '0000101', '0000110', '0000111', '0001000', '0001001', '0001010', '0001011', '0001100', '0001101', '0001110', '0001111', '0010000', '0010001', '0010010', '0010011', '0010100', '0010101', '0010110', '0010111', '0011000', '0011001', '0011010', '0011011', '0011100', '0011101', '0011110', '0011111', '0100000', '0100001', '0100010', '0100011', '0100100', '0100101', '0100110', '0100111', '0101000', '0101001', '0101010', '0101011', '0101100', '0101101', '0101110', '0101111', '0110000', '0110001', '0110010', '0110011', '0110100', '0110101', '0110110', '0110111', '0111000', '0111001', '0111010', '0111011', '0111100', '0111101', '0111110', '0111111', '1000000', '1000001', '1000010', '1000011', '1000100', '1000101', '1000110', '1000111', '1001000', '1001001', '1001010', '1001011', '1001100', '1001101', '1001110', '1001111', '1010000', '1010001', '1010010', '1010011', '1010100', '1010101', '1010110', '1010111', '1011000', '1011001', '1011010'

In [3]:
 #Extra options
#reg_coeffs = reg_coeffs,draw = [states_draw_list,states_draw_names] ,forbidden = states_forbidden_list,multi_mode = multi_mode,
#maxA = ops_max_amp,use_gpu = True
uks,Uf=Grape(H0,Hops,Hnames,U,total_time,steps,psi0,convergence,penalty_coeffs = penalty_coeffs, use_gpu = True, maxA = ops_max_amp,
      draw = [states_draw_list,states_draw_names], show_plots = False, method = 'L-BFGS-B', unitary_error = 1e-4,switch=False, gate ='QFT')

TypeError: 'NoneType' object is not iterable

In [ ]:
np.save('QFT_'+str(qubit_num),uks)